<a href="https://colab.research.google.com/github/shaking54/CS114.K21.KHTN/blob/master/Sarcasm%5C%20Detection/Sarcasm_Headlines.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#*Sarcasm Dectection
https://www.kaggle.com/rmisra/news-headlines-dataset-for-sarcasm-detection

- Input bài toán: Một chuỗi string

- Output: Chuỗi string nhận được có mang tính Sarcastic hay không

####*Thu thập dữ liệu

- Dữ liệu cũ về Sarcasm dectection được lấy từ Twitters. Tuy nhiên, do dữ liệu này có độ nhiễu nên vẫn còn hạn chế. Để vượt qua được điểm này thì dữ liệu mới về Sarcasm detection được lấy từ 2 websites đó là https://www.theonion.com/ và https://www.huffpost.com . Về dữ liệu mang tính Sarcastic thì được lấy từ trang web TheOnion ở trong mục News in Brief và News in Photos categories. Còn dữ liệu không mang tính Sarcastic thì được lấy từ trang web HuffPost.


In [127]:
!wget https://raw.githubusercontent.com/shaking54/CS114.K21.KHTN/master/Sarcasm%20Detection/Sarcasm_Headlines_Dataset.json

--2020-06-25 07:55:19--  https://raw.githubusercontent.com/shaking54/CS114.K21.KHTN/master/Sarcasm%20Detection/Sarcasm_Headlines_Dataset.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5616830 (5.4M) [text/plain]
Saving to: ‘Sarcasm_Headlines_Dataset.json.1’

Sarcasm_Headlines_D 100%[===================>]   5.36M  20.3MB/s    in 0.3s    

2020-06-25 07:55:20 (20.3 MB/s) - ‘Sarcasm_Headlines_Dataset.json.1’ saved [5616830/5616830]



In [128]:
import pandas as pd
data=pd.read_json('/content/Sarcasm_Headlines_Dataset.json', lines=True) #Load dataset from json file
data.head() #get 5 samples from dataset

,article_link,headline,is_sarcastic
0,https://www.huffingtonpost.com/entry/versace-b...,former versace store clerk sues over secret 'b...,0
1,https://www.huffingtonpost.com/entry/roseanne-...,the 'roseanne' revival catches up to our thorn...,0
2,https://local.theonion.com/mom-starting-to-fea...,mom starting to fear son's web series closest ...,1
3,https://politics.theonion.com/boehner-just-wan...,"boehner just wants wife to listen, not come up...",1
4,https://www.huffingtonpost.com/entry/jk-rowlin...,j.k. rowling wishes snape happy birthday in th...,0


In [129]:
sarcasm = data.drop(columns='article_link') # drop the column article_link

##*Xử lý dữ liệu

- Các headline sẽ được chuyển về lowercase, loại bỏ các ký tự đặc biệt.
- Sau đó sẽ phân chia dữ liệu train và dữ liệu test
- Khai báo Vectorize để biểu diễn dữ liệu train dưới dạng vector và tạo một từ điển từ dữ liệu train
- Sử dụng lại từ điển của dữ liệu trên để biểu diễn dữ liệu test dưới dạng vector

In [130]:
import re
def pre_process(headline):
	headline = headline.lower() # convert the headline to lower case                                      
	headline = re.sub("&lt;/?.*?&gt;"," &lt;&gt; ", headline) # remove 
	headline = re.sub("(\\d|\\WZhe Yang)+"," ", headline)                   
	return headline

sarcasm['headline'] = sarcasm['headline'].apply(lambda x:pre_process(x))
sarcasm['headline']

0        former versace store clerk sues over secret 'b...
1        the 'roseanne' revival catches up to our thorn...
2        mom starting to fear son's web series closest ...
3        boehner just wants wife to listen, not come up...
4        j.k. rowling wishes snape happy birthday in th...
                               ...                        
26704                 american politics in moral free-fall
26705                               america's best   hikes
26706                                reparations and obama
26707    israeli ban targeting boycott supporters raise...
26708                       gourmet gifts for the foodie  
Name: headline, Length: 26709, dtype: object

In [131]:
from sklearn.model_selection import train_test_split
X = sarcasm['headline']
y = sarcasm['is_sarcastic']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3) #split train set and test set

In [132]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

vectorizer1 = CountVectorizer(analyzer='word',ngram_range=(1,1), max_features=10000)
X_train = vectorizer1.fit_transform(X_train)

vectorizer2 = CountVectorizer(vocabulary=vectorizer1.vocabulary_,ngram_range=(1,1))
X_test = vectorizer2.fit_transform(X_test)

In [133]:
X_train.shape

(18696, 10000)

##Chọn Model để train dữ liệu
- Sử dụng các thuật toán như Suport vector classifications, Decision Tree classifications, Random Forest Classification, Voting Classification.


In [134]:
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
SVClassifiers = SVC()
SVClassifiers.fit(X_train, y_train, )
SVC_predict = SVClassifiers.predict(X_test)
SCV_score = accuracy_score(y_test, SVC_predict)
SCV_score

0.8398851865718208

In [135]:
from sklearn.svm import LinearSVC

LinearSVClassifier = LinearSVC()
LinearSVClassifier.fit(X_train, y_train)
Linear_predict = LinearSVClassifier.predict(X_test)
Linearscore = accuracy_score(y_test, Linear_predict)
Linearscore 

0.8154249344814676

In [136]:
from sklearn.tree import DecisionTreeClassifier

Decisionclassifier = DecisionTreeClassifier()
Decisionclassifier.fit(X_train, y_train)
Decisionclassifier_predict = Decisionclassifier.predict(X_test)
Decisionclassifier_score = accuracy_score(y_test, Decisionclassifier_predict)
Decisionclassifier_score

0.7434169474603769

In [137]:
from sklearn.ensemble import RandomForestClassifier
RFClassifier = RandomForestClassifier()
RFClassifier.fit(X_train, y_train)
RFClassifier_predict = RFClassifier.predict(X_test)
RFClassifier_score = accuracy_score(y_test, RFClassifier_predict)
RFClassifier_score
print(RFClassifier.score(X_test, y_test))

0.8000748783227256


In [138]:
# from sklearn.ensemble import VotingClassifier
# from sklearn.svm import SVC
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.tree import DecisionTreeClassifier
# RFClassifier = RandomForestClassifier()
# SVClassifiers = SVC()
# Decisionclassifier = DecisionTreeClassifier()
# voting_clf = VotingClassifier(
#         estimators=[('svc', clf), ('rf', random_clf), ('dc', tree_clf)],
#         voting='hard'
# )

#* Xử lý dữ liệu mới.
### crawl dữ liệu mới từ 2 nguồn là Huffpost và TheOnion bằng scapper extension của google chrome
- Sau khi crawl dữ liệu thì được 2 file .txt
- Tiếp theo load 2 file .txt vừa crawl được lên google colab bằng pandas
- Gán labels cho dữ liệu từ Huffpost là 0 và TheOnion bằng 1
- Chuyển 2 bảng dữ liệu thành dataframe bằng pandas
- Gộp 2 bảng lại và lưu xuống file json


In [139]:
# import pandas as pd

# onion = pd.read_csv('newOnion.txt', sep='\n')
# onion['is_sarcastic'] = 1
# huff = pd.read_csv('Huffpost.txt', sep='\n')
# huff['is_sarcastic'] = 0
# onion = pd.DataFrame(data=onion)
# huff = pd.DataFrame(data=huff)
# onion = onion.append(huff, sort=1)
# onion.sample(frac=1)
# onion.to_json('headline_data.json', orient='records', lines=True)

#* Chạy Model đã train với dữ liệu mới tạo được
- Model Support vector machine classifications
    - performance được 80.9% so với trên dữ liệu test là xấp xỉ 83%

- Model Decision Tree classifications
    - Performance được 68.5 % so với xấp xỉ 75% với dữ liệu test

- Model Random Forest Classifications
    - Performance được 73.3 % so với xấp xỉ 81% trên dữ liệu test



In [140]:
!wget https://raw.githubusercontent.com/shaking54/CS114.K21.KHTN/master/Sarcasm%20Detection/headline_data.json

--2020-06-25 07:57:22--  https://raw.githubusercontent.com/shaking54/CS114.K21.KHTN/master/Sarcasm%20Detection/headline_data.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 229984 (225K) [text/plain]
Saving to: ‘headline_data.json.1’

headline_data.json. 100%[===================>] 224.59K  --.-KB/s    in 0.05s   

2020-06-25 07:57:22 (4.45 MB/s) - ‘headline_data.json.1’ saved [229984/229984]



In [141]:
newdata = pd.read_json('/content/headline_data.json', lines=True)
newdata.sample(frac=1)

,headline,is_sarcastic
1300,"In An Election Year, 2 Vulnerable GOP Senators...",0
1117,ACLU Drags John Bolton's Record After He Retwe...,0
166,"Look, Brother, Sister Don’t Like That They’re ...",1
1525,NYC Mayor Bill De Blasio Enforces Curfew Again...,0
1651,Fox News Legal Analyst Delivers Withering Crit...,0
...,...,...
629,"‘I Could Spare Some Change,’ Says Man About To...",1
963,Teary Andrew Luck Describes Fighting Through S...,1
1340,Anna Wintour Apologizes For Race-Related 'Mist...,0
679,"‘New Day, Same Bullshit,’ Whispers Dalai Lama ...",1


In [142]:
newdata['headline'] = newdata['headline'].apply(lambda x:pre_process(x))
Xtest = newdata['headline']
ytest = newdata['is_sarcastic']

Xtest = vectorizer2.fit_transform(Xtest)

In [143]:
for cls in [SVClassifiers,LinearSVClassifier, Decisionclassifier,RFClassifier]:
    yscore = cls.score(Xtest, ytest)
    print(cls.__class__.__name__, yscore)

SVC 0.7955
LinearSVC 0.7715
DecisionTreeClassifier 0.6625
RandomForestClassifier 0.737


#*Sử dụng model để dự đoán một  đoạn headline bất kì

In [144]:
headline = input("Enter headline: ")
for cls in [SVClassifiers, LinearSVClassifier,Decisionclassifier,RFClassifier]:
    print(cls.__class__.__name__, cls.predict(vectorizer2.transform([pre_process(headline)])))

Enter headline: Thu Nga stupid
SVC [1]
LinearSVC [1]
DecisionTreeClassifier [1]
RandomForestClassifier [1]


In [145]:
!wget https://raw.githubusercontent.com/duongcscx/CS114.K21.KHTN/master/huffpost.json
!wget https://raw.githubusercontent.com/duongcscx/CS114.K21.KHTN/master/theonion.json

--2020-06-25 07:58:17--  https://raw.githubusercontent.com/duongcscx/CS114.K21.KHTN/master/huffpost.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 160023 (156K) [text/plain]
Saving to: ‘huffpost.json.1’

huffpost.json.1     100%[===================>] 156.27K  --.-KB/s    in 0.04s   

2020-06-25 07:58:17 (3.81 MB/s) - ‘huffpost.json.1’ saved [160023/160023]

--2020-06-25 07:58:19--  https://raw.githubusercontent.com/duongcscx/CS114.K21.KHTN/master/theonion.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 309885 (303K) [text/plain]


In [146]:
theonion = pd.read_json('/content/theonion.json', lines=True)
Duongdata = pd.read_json('/content/huffpost.json', lines=True)
Duongdata = pd.DataFrame(data=Duongdata)
Duongdata = Duongdata.append(theonion)
Duongdata=Duongdata.drop(columns='article_link')


In [147]:
Duongdata.head()
len(Duongdata)

2000

In [148]:
from sklearn.metrics import classification_report
Duongdata['headline'] = Duongdata['headline'].apply(lambda x:pre_process(x))
DuongX = Duongdata['headline']
Duongy = Duongdata['is_sarcastic']

DuongX = vectorizer2.fit_transform(DuongX)
for cls in [SVClassifiers,LinearSVClassifier, Decisionclassifier,RFClassifier]:
    Duongpred = cls.predict(DuongX)
    yscore = cls.score(DuongX, Duongy)
    print(classification_report(Duongy, Duongpred))
    print(cls.__class__.__name__, yscore)

              precision    recall  f1-score   support

           0       0.66      0.85      0.74       703
           1       0.90      0.76      0.83      1297

    accuracy                           0.79      2000
   macro avg       0.78      0.80      0.78      2000
weighted avg       0.82      0.79      0.80      2000

SVC 0.7915
              precision    recall  f1-score   support

           0       0.62      0.81      0.70       703
           1       0.87      0.73      0.80      1297

    accuracy                           0.76      2000
   macro avg       0.75      0.77      0.75      2000
weighted avg       0.79      0.76      0.76      2000

LinearSVC 0.7585
              precision    recall  f1-score   support

           0       0.50      0.76      0.60       703
           1       0.82      0.59      0.69      1297

    accuracy                           0.65      2000
   macro avg       0.66      0.68      0.65      2000
weighted avg       0.71      0.65      0.66   